In [ ]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
from pathlib import Path
from watch_recognition.reports import run_on_image_debug

In [ ]:
image_size = (224, 224)
model_path = "./models/keypoint/efficientnetb0/run_1631439816.02976.h5"
loaded_model = tf.keras.models.load_model(model_path, compile=False)
loaded_model.summary()

In [ ]:
for path in [
    Path("../example_data/Zrzut ekranu 2021-08-25 o 22.24.14.jpg"),
    Path("../example_data/Zrzut ekranu 2021-08-25 o 22.24.24.jpg"),
    Path("../example_data/test-image-2.jpg"),
]:
    test_image = tf.keras.preprocessing.image.load_img(
        path, "rgb", target_size=image_size, interpolation="bicubic",
    )
    test_image_np = tf.keras.preprocessing.image.img_to_array(test_image)

    run_on_image_debug(loaded_model, test_image_np)